In [2]:
#!pip install langchain openai chromadb unstructured tiktoken

In [7]:
#!pip install unstructured[pdf] pdf2image
#!pip install -U langchain-community

In [2]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [1]:
# 📚 Étape 1 – Chargement du fichier PDF (ou TXT)


from langchain.document_loaders import UnstructuredPDFLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader


# Remplace par ton chemin de fichier
#loader = UnstructuredPDFLoader("madame-bovary_Gustave-flaubert.pdf")
loader = PyPDFLoader("documents/madame-bovary_Gustave-flaubert.pdf")
docs = loader.load()

print(f"{len(docs)} pages chargées")

420 pages chargées


In [10]:
docs[0:4]

[Document(metadata={'producer': 'Acrobat Distiller 6.0 (Windows)', 'creator': 'Acrobat PDFMaker 6.0 pour Word', 'creationdate': '2004-12-29T12:46:28+01:00', 'moddate': '2004-12-29T12:47:49+01:00', 'title': 'MADAME BOVARY', 'comments': 'Publication en 1857', 'author': 'Gustave Flaubert', 'company': 'Ebooks libres et gratuits', 'source': 'documents/madame-bovary_Gustave-flaubert.pdf', 'total_pages': 420, 'page': 0, 'page_label': '1'}, page_content='Gustave Flaubert \nMADAME BOVARY \n(1857) \n \n \nÉdition du groupe « Ebooks libres et gratuits »'),
 Document(metadata={'producer': 'Acrobat Distiller 6.0 (Windows)', 'creator': 'Acrobat PDFMaker 6.0 pour Word', 'creationdate': '2004-12-29T12:46:28+01:00', 'moddate': '2004-12-29T12:47:49+01:00', 'title': 'MADAME BOVARY', 'comments': 'Publication en 1857', 'author': 'Gustave Flaubert', 'company': 'Ebooks libres et gratuits', 'source': 'documents/madame-bovary_Gustave-flaubert.pdf', 'total_pages': 420, 'page': 1, 'page_label': '2'}, page_conten

In [3]:
print(docs[0].page_content[:])

Gustave Flaubert 
MADAME BOVARY 
(1857) 
 
 
Édition du groupe « Ebooks libres et gratuits »


In [3]:
print(docs[419].page_content)

– 420 – 
À propos de cette édition électronique 
 
Texte libre de droits. 
 
Corrections, édition, conversion informatique et publication par 
le groupe : 
 
Ebooks libres et gratuits 
 
http://fr.groups.yahoo.com/group/ebooksgratuits 
 
Adresse du site web du groupe : 
 http://www.ebooksgratuits.com/ 
 
—— 
27 août 2003 
—— 
 
– Dispositions : 
Les livres que nous mettons à votre disposition, sont des textes li-
bres de droits, que vous pouvez utiliser librement, à une fin non 
commerciale et non professionnelle. Si vous désirez les faire para-
ître sur votre site, ils ne doivent être altérés en aucune sorte. 
Tout lien vers notre site est bienvenu… 
 
– Qualité : 
Les textes sont livrés tels quels sans garantie de leur intégrité par-
faite par rapport à l'original. Nous rappelons que c'est un travail 
d'amateurs non rétribués et nous essayons de promouvoir la 
culture littéraire avec de maigres moyens. 
 
Votre aide est la bienvenue ! 
 
VOUS POUVEZ NOUS AIDER 
À CONTRIBUER À FAIRE C

## Text Pre-Processing

In [4]:
from langchain.document_loaders import PyPDFLoader

# Charger le document
loader = PyPDFLoader("documents/madame-bovary_Gustave-flaubert.pdf")
docs = loader.load()

# Fusionner tout le texte dans un seul bloc pour faciliter la recherche
full_text = "\n".join([doc.page_content for doc in docs])

# Exemple simple pour diviser par parties en s'appuyant sur les titres visibles dans la table des matières
import re

# On cherche les grands titres de parties (PREMIÈRE PARTIE, DEUXIÈME PARTIE, TROISIÈME PARTIE, etc.)
chapitres = re.split(r"\n+(PREMIÈRE PARTIE|DEUXIÈME PARTIE|TROISIÈME PARTIE)\n+", full_text)

# Maintenant, on crée un dictionnaire pour stocker tout ça
chapitres_dict = {}
for i in range(1, len(chapitres), 2):
    partie = chapitres[i]
    contenu = chapitres[i+1]
    chapitres_dict[partie] = contenu.strip()

# Pour afficher les titres et un extrait de chaque partie
for titre, texte in chapitres_dict.items():
    print(f"{titre} : {texte[:200]}...\n")


In [11]:
#full_text
#chapitres

In [12]:
from langchain.document_loaders import PyPDFLoader

# Charger le PDF
loader = PyPDFLoader("documents/madame-bovary_Gustave-flaubert.pdf")
docs = loader.load()

# Mapping des pages par index (attention : index 0 = page 1 du PDF)
page_map = {doc.metadata["page"] + 1: doc.page_content for doc in docs}

# Table des matières extraite manuellement
table_matiere = {
    "PREMIÈRE PARTIE": {
        "I": 5, "II": 15, "III": 24, "IV": 30, "V": 35, "VI": 39, "VII": 44, "VIII": 51, "IX": 62
    },
    "DEUXIÈME PARTIE": {
        "I": 75, "II": 85, "III": 93, "IV": 105, "V": 109, "VI": 120, "VII": 137,
        "VIII": 148, "IX": 175, "X": 189, "XI": 200, "XII": 214, "XIII": 232, "XIV": 244, "XV": 256
    },
    "TROISIÈME PARTIE": {
        "I": 267, "II": 286, "III": 299, "IV": 302, "V": 306, "VI": 327, "VII": 349,
        "VIII": 368, "IX": 391, "X": 402, "XI": 409
    }
}

# Fonction pour extraire les chapitres
def extraire_chapitres(pages_dict, toc):
    result = {}
    for partie, chapitres in toc.items():
        partie_dict = {}
        titres = list(chapitres.keys())
        pages = list(chapitres.values())
        
        for i, titre in enumerate(titres):
            start_page = pages[i]
            end_page = pages[i+1] if i + 1 < len(pages) else None

            # Extraire le texte des pages concernées
            contenu = []
            for page_num in range(start_page, end_page if end_page else 420):
                if page_num in pages_dict:
                    contenu.append(pages_dict[page_num])
            partie_dict[titre] = "\n".join(contenu).strip()
        
        result[partie] = partie_dict
    return result

# Exécution
chapitres_structures = extraire_chapitres(page_map, table_matiere)

# Exemple : Afficher un extrait du chapitre I de la PREMIÈRE PARTIE
print(chapitres_structures["PREMIÈRE PARTIE"]["I"][:1000])

– 5 – 
PREMIÈRE PARTIE 
 
I 
 
Nous étions à l’Étude, quand le  Proviseur entra, suivi d’un 
nouveau habillé en bourgeois et d’un garçon de classe qui portait 
un grand pupitre. Ceux qui dormaient se réveillèrent, et chacun 
se leva comme surpris dans son travail. 
 
Le Proviseur nous fit signe de nous rasseoir ; puis, se tour-
nant vers le maître d’études : 
 
– Monsieur Roger, lui dit-il à de mi-voix, voici un élève que je 
vous recommande, il entre en cinquième. Si son travail et sa 
conduite sont méritoires, il passera dans les grands, où l’appelle 
son âge. 
 
Resté dans l’angle, derrière la porte, si bien qu’on l’apercevait 
à peine, le nouveau était un gars de la campagne, d’une quinzaine 
d’années environ, et plus haut de taille qu’aucun de nous tous. Il 
avait les cheveux coupés droit sur le front, comme un chantre de 
village, l’air raisonnable et fort embarrassé. Quoiqu’il ne fût pas 
large des épaules, son habit-veste de drap vert à boutons noirs 
devait le gêner aux entour

In [23]:
print(chapitres_structures["PREMIÈRE PARTIE"]["I"])

– 5 – 
PREMIÈRE PARTIE 
 
I 
 
Nous étions à l’Étude, quand le  Proviseur entra, suivi d’un 
nouveau habillé en bourgeois et d’un garçon de classe qui portait 
un grand pupitre. Ceux qui dormaient se réveillèrent, et chacun 
se leva comme surpris dans son travail. 
 
Le Proviseur nous fit signe de nous rasseoir ; puis, se tour-
nant vers le maître d’études : 
 
– Monsieur Roger, lui dit-il à de mi-voix, voici un élève que je 
vous recommande, il entre en cinquième. Si son travail et sa 
conduite sont méritoires, il passera dans les grands, où l’appelle 
son âge. 
 
Resté dans l’angle, derrière la porte, si bien qu’on l’apercevait 
à peine, le nouveau était un gars de la campagne, d’une quinzaine 
d’années environ, et plus haut de taille qu’aucun de nous tous. Il 
avait les cheveux coupés droit sur le front, comme un chantre de 
village, l’air raisonnable et fort embarrassé. Quoiqu’il ne fût pas 
large des épaules, son habit-veste de drap vert à boutons noirs 
devait le gêner aux entour

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
chunks = splitter.split_documents(docs)

print(f"{len(chunks)} chunks générés.")

513 chunks générés.


In [18]:
import sqlite3
print(sqlite3.sqlite_version)

3.31.1


In [8]:

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embedding_function = OpenAIEmbeddings(
    #api_key=OPENAI_API_KEY,
    model="text-embedding-3-small")

# ChromaDB en local (dossier "chroma_db")
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_function,
    persist_directory="./chroma_db"
)

# Sauvegarde pour usage ultérieur
vectorstore.persist()

/tmp/ipykernel_1706/4028485364.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_function = OpenAIEmbeddings(


NameError: name 'chunks' is not defined

In [14]:
#!pip install faiss-cpu
#!pip install langchain_openai

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

#from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# Chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
chunks = splitter.split_documents(docs)
print(f"{len(chunks)} chunks générés.")

# Embeddings
embeddings = OpenAIEmbeddings(
    #api_key=OPENAI_API_KEY,
    model="text-embedding-3-small")
vectorstore = FAISS.from_documents(chunks, embeddings)

513 chunks générés.


In [6]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    model="gpt-4o", 
    temperature=0)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"  # pour commencer simple
)

# Question exemple
query = "Fais un résumé du chapitre 1 du livre"
result = qa_chain.run(query)
print(result)

Le chapitre 1 du livre commence par l'arrivée d'un nouveau élève dans une école. Le proviseur introduit ce nouvel élève, habillé en bourgeois, à la classe et au maître d'études, Monsieur Roger. Le nouvel élève, qui vient de la campagne, est décrit comme étant plus grand que les autres élèves, avec une apparence modeste et un air embarrassé. Il écoute attentivement les leçons sans oser se détendre. À la fin de la journée, le maître d'études doit lui rappeler de se joindre aux autres élèves pour sortir.


In [51]:
query = "Qui est le nouvel élève, garçon de la campagne d'environ quinze ans dont l'auteur parle au chapitre 1 ?"
result = qa_chain.invoke(query)
print(result)

{'query': "Qui est le nouvel élève, garçon de la campagne d'environ quinze ans dont l'auteur parle au chapitre 1 ?", 'result': "Le nouvel élève, garçon de la campagne d'environ quinze ans dont l'auteur parle au chapitre 1, est Charles."}


In [15]:
query = "Extrait tous les passage dont on parle de l'age des personnages dans le Chapitre 1 ?"
result = qa_chain.run(query)
print(result)

Dans le Chapitre 1, il y a un passage où l'on mentionne l'âge d'un personnage :

"Resté dans l’angle, derrière la porte, si bien qu’on l’apercevait à peine, le nouveau était un gars de la campagne, d’une quinzaine d’années environ, et plus haut de taille qu’aucun de nous tous."


In [11]:
extrait_de_text ="""
 
Nous étions à l’Étude, quand le  Proviseur entra, suivi d’un 
nouveau habillé en bourgeois et d’un garçon de classe qui portait 
un grand pupitre. Ceux qui dormaient se réveillèrent, et chacun 
se leva comme surpris dans son travail. 
 
Le Proviseur nous fit signe de nous rasseoir ; puis, se tour-
nant vers le maître d’études : 
 
– Monsieur Roger, lui dit-il à de mi-voix, voici un élève que je 
vous recommande, il entre en cinquième. Si son travail et sa 
conduite sont méritoires, il passera dans les grands, où l’appelle 
son âge. 
 
Resté dans l’angle, derrière la porte, si bien qu’on l’apercevait 
à peine, le nouveau était un gars de la campagne, d’une quinzaine 
d’années environ, et plus haut de taille qu’aucun de nous tous. Il 
avait les cheveux coupés droit sur le front, comme un chantre de 
village, l’air raisonnable et fort embarrassé. Quoiqu’il ne fût pas 
large des épaules, son habit-veste de drap vert à boutons noirs 
devait le gêner aux entournures et  laissait voir, par la fente des 
parements, des poignets rouges habitués à être nus. Ses jambes, 
en bas bleus, sortaient d’un. pantalon jaunâtre très tiré par les 
bretelles. Il était chaussé de souliers forts, mal cirés, garnis de 
clous. 
 
"""

In [24]:
query = "Qui est le nouveau habillé en bourgeois, dont l'auteur parle au chapitre 1 ?"
result = qa_chain.invoke(query)
print(result)

{'query': "Qui est le nouveau habillé en bourgeois, dont l'auteur parle au chapitre 1 ?", 'result': 'Le "nouveau" habillé en bourgeois dont l\'auteur parle au chapitre 1 est Charles Bovary. Il est décrit comme un élève qui, lors de son arrivée en classe, a du mal à articuler son nom, ce qui provoque les moqueries de ses camarades.'}


In [25]:
query = "Qui est le garçon de classe qui portait un grand pupitre, dont l'auteur parle au chapitre 1 ?"
result = qa_chain.invoke(query)
print(result)

{'query': "Qui est le garçon de classe qui portait un grand pupitre, dont l'auteur parle au chapitre 1 ?", 'result': "L'auteur ne donne pas de détails spécifiques sur le garçon de classe qui portait un grand pupitre dans le chapitre 1. Il est simplement mentionné comme accompagnant le nouveau élève et le Proviseur."}


In [30]:
query = """
{extrait_de_text}
Dans du chapitrre 1, le garçon de classe qui portait un grand pupitre mentionné est-it egalement un nouveau ? 
Sinon, aide-moi à mieux comprendre car je suis un peu embrouillé là

"""
result = qa_chain.run(query)
print(result)

Non, le garçon de classe qui portait un grand pupitre n'est pas un nouveau. Dans le contexte du chapitre 1, le garçon de classe accompagne le Proviseur et le nouveau élève, mais il n'est pas lui-même un nouvel élève. Le nouveau élève est celui qui est habillé en bourgeois et qui est recommandé par le Proviseur au maître d'études. Le garçon de classe semble être un élève déjà présent dans l'école, chargé d'apporter le pupitre pour le nouveau.


In [ ]:
query = """
{extrait_de_text}
Dans du chapitrre 1, c'est quoi le grand pupitre mentionné ?

"""
result = qa_chain.run(query)
print(result)

In [8]:
query = """
{extrait_de_text}
Dans du chapitrre 1, quel age à Charles bovary ?

"""
result = qa_chain.run(query)
print(result)

NameError: name 'qa_chain' is not defined

In [20]:
from langchain.prompts import PromptTemplate

character_extraction_prompt = PromptTemplate(
    input_variables=["context"],
    template="""
Tu es un assistant littéraire expert en analyse de romans.

Voici un extrait de livre :

-------------------
{context}
-------------------

Ta tâche est d'identifier **tous les personnages mentionnés dans ce passage**, même brièvement.
Pour chaque personnage :
- Donne **le nom complet** (ou partiel s'il n'y a que cela)
- Indique **un court résumé de son rôle ou importance** dans le récit (si possible)
- Mentionne les relations ou interactions importantes s’il y en a

Formate ta réponse comme ceci :

1. Nom : ...
   Rôle : ...
   Liens : ...

2. Nom : ...
   ...

Ne fais **aucune supposition** en dehors du texte donné. Si aucun personnage n’est mentionné, dis "Aucun personnage identifié."
"""
)


In [19]:
context = chunks[0:20]
context

[Document(metadata={'producer': 'Acrobat Distiller 6.0 (Windows)', 'creator': 'Acrobat PDFMaker 6.0 pour Word', 'creationdate': '2004-12-29T12:46:28+01:00', 'moddate': '2004-12-29T12:47:49+01:00', 'title': 'MADAME BOVARY', 'comments': 'Publication en 1857', 'author': 'Gustave Flaubert', 'company': 'Ebooks libres et gratuits', 'source': 'documents/madame-bovary_Gustave-flaubert.pdf', 'total_pages': 420, 'page': 0, 'page_label': '1'}, page_content='Gustave Flaubert \nMADAME BOVARY \n(1857) \n \n \nÉdition du groupe « Ebooks libres et gratuits »'),
 Document(metadata={'producer': 'Acrobat Distiller 6.0 (Windows)', 'creator': 'Acrobat PDFMaker 6.0 pour Word', 'creationdate': '2004-12-29T12:46:28+01:00', 'moddate': '2004-12-29T12:47:49+01:00', 'title': 'MADAME BOVARY', 'comments': 'Publication en 1857', 'author': 'Gustave Flaubert', 'company': 'Ebooks libres et gratuits', 'source': 'documents/madame-bovary_Gustave-flaubert.pdf', 'total_pages': 420, 'page': 1, 'page_label': '2'}, page_conten

In [42]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    model="gpt-4o", temperature=0)
character_chain = LLMChain(llm=llm, prompt=character_extraction_prompt)

# Exemple avec un chunk du livre
result = character_chain.run(context=context)
print(result)


1. Nom : Gustave Flaubert
   Rôle : Auteur du livre "Madame Bovary"
   Liens : Aucun lien direct avec l'intrigue du récit, mentionné en tant qu'auteur.

2. Nom : Marie-Antoine-Jules Senard
   Rôle : Dédicataire du livre, membre du barreau de Paris, ex-président de l'Assemblée nationale et ancien ministre de l'Intérieur
   Liens : Aucun lien direct avec l'intrigue du récit, mentionné dans la dédicace.

3. Nom : Louis Bouilhet
   Rôle : Mentionné dans la dédicace
   Liens : Aucun lien direct avec l'intrigue du récit, mentionné dans la dédicace.

4. Nom : Charles Bovary
   Rôle : Personnage principal, élève introduit dans le récit, plus tard médecin
   Liens : Fils de M. Charles-Denis-Bartholomé Bovary et de Madame Bovary (mère), marié à Madame Dubuc

5. Nom : M. Charles-Denis-Bartholomé Bovary
   Rôle : Père de Charles Bovary, ancien aide-chirurgien-major
   Liens : Père de Charles Bovary, mari de Madame Bovary (mère)

6. Nom : Madame Bovary (mère)
   Rôle : Mère de Charles Bovary, épous

In [21]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    model="gpt-4.1", temperature=0)
character_chain = LLMChain(llm=llm, prompt=character_extraction_prompt)

# Exemple avec un chunk du livre
result = character_chain.run(context=context)
print(result)

/tmp/ipykernel_3040/2078089577.py:7: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  character_chain = LLMChain(llm=llm, prompt=character_extraction_prompt)


1. Nom : Gustave Flaubert  
   Rôle : Auteur du roman (mentionné dans la dédicace et la préface)  
   Liens : Aucun lien direct avec l’intrigue, mais il dédie le livre à d’autres personnes.

2. Nom : Marie-Antoine-Jules Senard  
   Rôle : Destinataire de la dédicace, membre du barreau de Paris, ex-président de l’Assemblée nationale, ancien ministre de l’Intérieur  
   Liens : Soutien de Flaubert lors de la publication du livre.

3. Nom : Louis Bouilhet  
   Rôle : Personne à qui le livre est dédié  
   Liens : Aucun lien direct avec l’intrigue.

4. Nom : Monsieur Roger  
   Rôle : Maître d’études à l’école où se déroule la scène d’ouverture  
   Liens : Reçoit le nouvel élève recommandé par le Proviseur.

5. Nom : Le Proviseur  
   Rôle : Chef d’établissement scolaire, introduit le nouveau dans la classe  
   Liens : Présente Charles Bovary à Monsieur Roger et à la classe.

6. Nom : Charles Bovary (appelé aussi "le nouveau", "Charbovari")  
   Rôle : Personnage principal du passage, no

In [23]:
context = chunks[20:30]
context

from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    model="gpt-4.1", temperature=0)
character_chain = LLMChain(llm=llm, prompt=character_extraction_prompt)

# Exemple avec un chunk du livre
result = character_chain.run(context=context)
print(result)

1. Nom : Charles Bovary (souvent appelé simplement "Charles" ou "M. Bovary")
   Rôle : Médecin de campagne, personnage principal du passage, appelé à la ferme des Bertaux pour soigner une jambe cassée.
   Liens : Mari de "Madame Bovary jeune", médecin du père Rouault, rencontre et interactions avec Emma Rouault.

2. Nom : Madame Bovary jeune
   Rôle : Épouse de Charles Bovary, inquiète pour son mari lors de son départ nocturne.
   Liens : Femme de Charles Bovary.

3. Nom : Nastasie
   Rôle : Bonne (servante) présente lors de la lecture de la lettre, tient la lumière.
   Liens : Employée chez les Bovary.

4. Nom : Valet d’écurie
   Rôle : Employé chargé de partir en avant pour préparer le chemin de Charles vers la ferme des Bertaux.
   Liens : Employé chez les Bovary.

5. Nom : Gamin (jeune garçon)
   Rôle : Enfant envoyé à la rencontre de Charles pour lui montrer le chemin de la ferme et ouvrir les clôtures.
   Liens : Guide Charles jusqu’aux Bertaux.

6. Nom : M. Rouault
   Rôle : Cul

In [43]:
from langchain.prompts import PromptTemplate

summary_template = PromptTemplate(
    input_variables=["chapter_title"],
    template="""
Tu es un expert en littérature. Tu lis un extrait de livre et ton objectif est de **résumer le chapitre** intitulé : "{chapter_title}".

Consignes :
- Résume en **français clair**.
- Détaille les **idées principales**, les **événements clés**, et les **personnages impliqués**.
- Sois concis mais informatif (environ 150-300 mots).

Voici le contenu extrait du chapitre :
{context}

Résumé :
""",
)


In [46]:
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    model="gpt-4o", temperature=0)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": summary_template}
)

#query = "Fais un résumé du chapitre 1 du livre "
response = qa_chain.run({"chapter_title": "Chapitre 1"})
print(response)


ValueError: Missing some input keys: {'query'}

In [24]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

from langchain.prompts import PromptTemplate

chapter_summary_prompt = PromptTemplate(
    input_variables=["context", "chapter_title"],
    template="""
Tu es un expert en littérature. Tu dois résumer un chapitre intitulé **"{chapter_title}"** à partir du contenu suivant :

{context}

Consignes :
- Résume en français clair et structuré.
- Propose un titre qui resume bien le chapitre
- Mentionne les points importants, événements clés, idées fortes et personnages principaux.
- Garde un ton neutre, professionnel et informatif.
- environ 1000-1500 mots

Résumé :
""",
)




# Ton LLM
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model="gpt-4o-mini", temperature=0)

# Chaîne de résumé de chapitre
summary_chain = LLMChain(llm=llm, prompt=chapter_summary_prompt)

# 🔍 Recherche vectorielle avec ta base
chapter_title = "Chapitre 1"
docs = retriever.get_relevant_documents(chapter_title)

# 👇 Concatène ou join le contexte si tu veux
context = "\n\n".join([doc.page_content for doc in docs])

# 🧠 Génère le résumé
result = summary_chain.run(context=context, chapter_title=chapter_title)
print(result)


/tmp/ipykernel_3040/2922314779.py:35: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(chapter_title)


### Titre : **L'Arrivée du Nouveau Élève et la Vie au Collège**

**Résumé du Chapitre 1 :**

Le premier chapitre de l'œuvre s'ouvre sur une scène typique d'une salle de classe, où les élèves sont en pleine étude. L'atmosphère est soudainement perturbée par l'entrée du Proviseur, accompagné d'un nouvel élève et d'un camarade de classe portant un grand pupitre. Ce moment marque l'introduction d'un personnage central, le nouveau, qui est décrit comme un jeune garçon de la campagne, âgé d'environ quinze ans. Sa stature est plus grande que celle de ses camarades, et son apparence vestimentaire, bien que modeste, est soigneusement décrite, soulignant son origine rurale et son adaptation à un environnement scolaire urbain.

Le Proviseur présente le nouvel élève au maître d'études, Monsieur Roger, en lui recommandant de surveiller son travail et sa conduite. Ce geste souligne l'importance de l'intégration et de la performance académique dans le cadre scolaire. Le nouveau élève, bien que timide

In [29]:
context = chunks[0:20]

from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

chapter_summary_prompt = PromptTemplate(
    input_variables=["context"],
    template="""
Tu es un expert en littérature. Tu dois résumer un chapitre à partir du contenu suivant :

{context}

Consignes :
- Résume en français clair et structuré.
- Propose un titre qui resume bien le chapitre
- Mentionne les points importants, événements clés, idées fortes et personnages principaux.
- Garde un ton neutre, professionnel et informatif.
- environ 500 mots

Résumé :
""",
)

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    model="gpt-4.1", temperature=0)
chapter_summary = LLMChain(llm=llm, prompt=chapter_summary_prompt)

# Exemple avec un chunk du livre
result = chapter_summary.run(context=context)
print(result)

**Titre : Les Origines et la Jeunesse de Charles Bovary**

**Résumé du premier chapitre de "Madame Bovary" de Gustave Flaubert**

Le premier chapitre de "Madame Bovary" s’ouvre sur l’arrivée d’un nouvel élève dans une classe de collège : Charles Bovary. La scène, racontée à la première personne du pluriel, plonge le lecteur dans l’ambiance scolaire, marquée par la curiosité et la moquerie des élèves envers ce garçon maladroit, vêtu de façon provinciale et manifestement peu à l’aise dans ce nouvel environnement. L’épisode de la casquette, objet ridicule et source de railleries, symbolise d’emblée la gaucherie et la timidité de Charles, qui peine à s’intégrer.

Le professeur, après avoir calmé la classe, tente d’obtenir le nom du nouveau, mais Charles, intimidé, le prononce de façon incompréhensible, déclenchant un chahut général. Finalement, son nom, Charles Bovary, est compris et il est installé sur le banc des paresseux, déjà marqué par la différence et l’exclusion.

Le récit s’attard

In [28]:
context = chunks[0:20]

from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

chapter_summary_prompt = PromptTemplate(
    input_variables=["context"],
    template="""
Tu es un expert en littérature. Tu dois résumer un chapitre à partir du contenu suivant :

{context}

Consignes :
- Résume en français clair et structuré.
- Propose un titre qui resume bien le chapitre
- Mentionne les points importants, événements clés, idées fortes et personnages principaux.
- Garde un ton neutre, professionnel et informatif.
- environ 1000-1500 mots

Résumé :
""",
)

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    model="gpt-4.1", temperature=0)
chapter_summary = LLMChain(llm=llm, prompt=chapter_summary_prompt)

# Exemple avec un chunk du livre
result = chapter_summary.run(context=context)
print(result)

**Titre du chapitre :**  
**Les Origines et la Formation de Charles Bovary**

---

**Résumé du chapitre I de la Première Partie de "Madame Bovary"**

Ce premier chapitre de "Madame Bovary" de Gustave Flaubert s’attache à présenter le personnage de Charles Bovary, futur mari d’Emma, en retraçant son enfance, son éducation, ses origines familiales et ses premiers pas dans la vie adulte. À travers une narration précise et détaillée, Flaubert dresse le portrait d’un homme ordinaire, sans éclat, dont la médiocrité et la passivité vont marquer l’ensemble du roman.

---

### 1. **L’arrivée de Charles au collège : la scène inaugurale**

Le récit s’ouvre sur une scène de classe, vue à la première personne du pluriel, qui plonge le lecteur dans l’ambiance d’un collège de province. Un nouveau venu, Charles Bovary, fait son entrée, accompagné du proviseur et d’un garçon de classe. Son apparence rustique, sa maladresse et sa timidité le distinguent immédiatement des autres élèves. Son habit mal aju

In [27]:
context = chunks[20:30]

from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI



chapter_summary_prompt = PromptTemplate(
    input_variables=["context"],
    template="""
Tu es un expert en littérature. Tu dois résumer un chapitre à partir du contenu suivant :

{context}

Consignes :
- Résume en français clair et structuré.
- Propose un titre qui resume bien le chapitre
- Mentionne les points importants, événements clés, idées fortes et personnages principaux.
- Garde un ton neutre, professionnel et informatif.
- environ 1000-1500 mots

Résumé :
""",
)

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    model="gpt-4.1", temperature=0)
chapter_summary = LLMChain(llm=llm, prompt=chapter_summary_prompt)

# Exemple avec un chunk du livre
result = chapter_summary.run(context=context)
print(result)

**Titre du chapitre :**  
**La rencontre de Charles Bovary et d’Emma Rouault aux Bertaux**

---

**Résumé détaillé :**

Ce chapitre de *Madame Bovary* de Gustave Flaubert marque un tournant décisif dans la vie de Charles Bovary, jeune officier de santé récemment marié, et introduit pour la première fois de façon significative le personnage d’Emma Rouault, future Madame Bovary. Il s’agit d’un chapitre charnière, où se nouent les premiers liens entre Charles et Emma, dans le cadre rural de la ferme des Bertaux.

---

### 1. **L’appel aux Bertaux : le départ de Charles**

L’action débute alors que Charles Bovary, installé à Tostes, reçoit en pleine nuit une lettre urgente. Celle-ci, cachetée de cire bleue, le supplie de se rendre sans délai à la ferme des Bertaux, où le propriétaire, M. Rouault, s’est cassé la jambe. La distance est importante (six lieues), la nuit noire, et la jeune épouse de Charles, inquiète, prend des précautions pour la sécurité de son mari : un valet d’écurie part e

In [19]:



from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

from langchain.prompts import PromptTemplate

chapter_summary_prompt = PromptTemplate(
    input_variables=["context", "chapter_title"],
    template="""
Tu es un expert en littérature. Tu dois résumer un chapitre intitulé **"{chapter_title}"** à partir du contenu suivant :

{context}

Consignes :
- Résume en français clair et structuré.
- Propose un titre qui resume bien le chapitre
- Mentionne les points importants, événements clés, idées fortes et personnages principaux.
- Garde un ton neutre, professionnel et informatif.
- environ 1000-1500 mots

Résumé :
""",
)

context = chunks[20:30]




# Ton LLM
llm = ChatOpenAI(
                #api_key=OPENAI_API_KEY,
                 model="gpt-4o-mini", temperature=0)

# Chaîne de résumé de chapitre
summary_chain = LLMChain(llm=llm, prompt=chapter_summary_prompt)

# 🔍 Recherche vectorielle avec ta base
chapter_title = "Chapitre 2"
docs = retriever.get_relevant_documents(chapter_title)

# 👇 Concatène ou join le contexte si tu veux
context = "\n\n".join([doc.page_content for doc in docs])

# 🧠 Génère le résumé
result = summary_chain.run(context=context, chapter_title=chapter_title)
print(result)

### Titre : **L'Arrivée du Nouveau Élève et les Premiers Échos de la Vie Sociale**

**Résumé du Chapitre 2 :**

Le chapitre 2 s'ouvre sur une scène dans une salle de classe où le Proviseur fait son entrée, accompagné d'un nouvel élève, un jeune garçon de la campagne, qui vient d'être admis en cinquième. Ce moment marque le début de l'intégration de ce nouvel élève dans un environnement scolaire qui lui est étranger. Le Proviseur présente le jeune homme au maître d'études, Monsieur Roger, en soulignant que si son comportement et ses résultats sont satisfaisants, il pourra passer dans la classe supérieure, celle des "grands".

Le nouveau, décrit comme un adolescent de quinze ans, se distingue par sa taille et son apparence rustique. Il est vêtu d'un habit de drap vert, qui semble inconfortable, et ses vêtements témoignent de ses origines modestes. Son attitude est celle d'un élève attentif, presque craintif, qui écoute avec une grande concentration les leçons, sans oser se relâcher ou se